# DX 704 Week 1 Project

This week's project will build a portfolio risk and return model, and make investing recommendations for hypothetical clients.
You will collect historical data, estimate returns and risks, construct efficient frontier portfolios, and sanity check the certainty of the maximum return portfolio.

The full project description and a template notebook are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-01


Feel free to use optimization tools or libraries (such as CVXOPT or scipy.optimize) to perform any calculations required for this mini project.

### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Data

Collect historical monthly price data for the last 24 months covering 6 different stocks.
The data should cover 24 consecutive months including the last month that ended before this week's material was released on Blackboard.
To be clear, if a month ends between the Blackboard release and submitting your project, you do not need to add that month.

The six different stocks must include AAPL, SPY and TSLA.
At least one of the remaining 3 tickers must start with the same letter as your last name (e.g. professor Considine could use COIN).
This is to encourage diversity in what stocks you analyze; if you discuss this project with classmates, please make sure that you pick different tickers to differentiate your work.
Do not pick stocks with fewer than 24 consecutive months of price data.

In [1]:
# YOUR CHANGES HERE
import yfinance as yf
import pandas as pd

tickers = ["AAPL", "SPY", "TSLA", "YETI", "MSFT", "NVDA"]

historical_prices = None

for t in tickers:
    ticker = yf.Ticker(t)
    history = ticker.history(period="2y")
    
   
    monthly_prices = history['Close'].resample('ME').last()
    
    if historical_prices is None:
        historical_prices = monthly_prices
    else:
        historical_prices = pd.concat([historical_prices, monthly_prices], axis=1)

historical_prices.columns = tickers
historical_prices.index = pd.to_datetime(historical_prices.index).strftime("%Y-%m-%d")

historical_prices = historical_prices.tail(24)
print("Historical Prices Data:")
print(historical_prices)

Historical Prices Data:
                  AAPL         SPY        TSLA       YETI        MSFT  \
Date                                                                    
2023-10-31  169.113556  408.794373  200.839996  42.520000  333.090302   
2023-11-30  188.355316  446.135223  240.080002  42.639999  374.042297   
2023-12-31  190.913666  466.503632  248.479996  51.779999  371.209137   
2024-01-31  182.851898  473.933411  187.289993  43.970001  392.472382   
2024-02-29  179.461273  498.666534  201.880005  41.040001  409.081238   
2024-03-31  170.257355  514.973938  175.789993  38.549999  416.083130   
2024-04-30  169.115555  494.210327  183.279999  35.720001  385.039124   
2024-05-31  191.138153  519.207275  178.080002  40.740002  411.295288   
2024-06-30  209.401917  537.524963  197.880005  38.150002  442.821350   
2024-07-31  220.795670  544.033936  232.070007  41.349998  414.485596   
2024-08-31  227.939163  556.745605  214.110001  40.320000  414.031647   
2024-09-30  231.920639  568

Save the data as a TSV file named "historical_prices.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
The date should be the last trading day of the month, so it may not be the last day of the month.
For example, the last trading day of November 2024 was 2024-11-29.
The remaining columns should contain the adjusted closing prices of the corresponding stock tickers on that day.


In [8]:
# YOUR CHANGES HERE
historical_prices.to_csv('historical_prices.tsv', sep='\t')
print("\nData saved to 'historical_prices.tsv'")


Data saved to 'historical_prices.tsv'


Submit "historical_prices.tsv" in Gradescope.

## Part 2: Calculate Historical Asset Returns

Calculate the historical asset returns based on the price data that you previously collected.

In [ ]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

historical_prices = pd.read_csv('historical_prices.tsv', sep='\t')

print("Columns in your data:", historical_prices.columns.tolist())
print("First few rows:")
print(historical_prices.head())

numeric_columns = historical_prices.select_dtypes(include=[np.number]).columns
print(f"\nNumeric columns found: {numeric_columns.tolist()}")

historical_returns = historical_prices[numeric_columns].pct_change().dropna()

print(f"\nHistorical returns calculated:")
print(f"Shape: {historical_returns.shape}")
print(historical_returns.head())

dates_for_returns = historical_prices['Date'].iloc[1:].reset_index(drop=True)

stock_columns = ['AAPL', 'SPY', 'TSLA', 'YETI', 'MSFT', 'NVDA']
returns_only = historical_returns[stock_columns].reset_index(drop=True)

returns_data = pd.concat([
    dates_for_returns.rename('date'),  
    returns_only
], axis=1)

print("\nFinal returns data with date column:")
print(returns_data.head())
print(f"Shape: {returns_data.shape}")
print(f"Columns: {returns_data.columns.tolist()}")

Columns in your data: ['Unnamed: 0', 'Date', 'AAPL', 'SPY', 'TSLA', 'YETI', 'MSFT', 'NVDA']
First few rows:
   Unnamed: 0        Date        AAPL         SPY        TSLA       YETI  \
0           0  2023-10-31  169.113556  408.794342  200.839996  42.520000   
1           1  2023-11-30  188.355301  446.135223  240.080002  42.639999   
2           2  2023-12-31  190.913666  466.503662  248.479996  51.779999   
3           3  2024-01-31  182.851913  473.933411  187.289993  43.970001   
4           4  2024-02-29  179.461273  498.666504  201.880005  41.040001   

         MSFT       NVDA  
0  333.090363  40.758278  
1  374.042236  46.745087  
2  371.209137  49.499973  
3  392.472412  61.499630  
4  409.081207  79.076805  

Numeric columns found: ['Unnamed: 0', 'AAPL', 'SPY', 'TSLA', 'YETI', 'MSFT', 'NVDA']

Historical returns calculated:
Shape: (23, 7)
   Unnamed: 0      AAPL       SPY      TSLA      YETI      MSFT      NVDA
1         inf  0.113780  0.091344  0.195379  0.002822  0.122945  0

Save the data as a TSV file named "historical_returns.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
Each row should have the date at the end of the month and the corresponding *relative* price changes.
For example, if the previous price was \$100 and the new price is \$110, the return value should be 0.10.
There should only be 23 rows of data in this file, since they are computed as the differences of 24 prices.

In [17]:
# YOUR CHANGES HERE

returns_data.to_csv('historical_returns.tsv', sep='\t', index=False, float_format='%.6f')
print("Historical returns saved to 'historical_returns.tsv'")

Historical returns saved to 'historical_returns.tsv'


Submit "historical_returns.tsv" in Gradescope.

## Part 3: Estimate Returns

Estimate the expected returns for each asset using the previously calculated return data.
Just compute the average (mean) return for each asset over your data set; do not use other estimators that have been mentioned.
This will serve as your estimate of expected return for each asset.

In [18]:
# YOUR CHANGES HERE

import pandas as pd

historical_returns_df = pd.read_csv('historical_returns.tsv', sep='\t')

print("Historical returns data loaded:")
print(f"Shape: {historical_returns_df.shape}")
print(historical_returns_df.head())

stock_columns = [col for col in historical_returns_df.columns if col != 'date']
print(f"\nStock columns: {stock_columns}")

mean_returns = historical_returns_df[stock_columns].mean()

print("\nCalculated mean returns:")
print(mean_returns)

estimated_returns_df = pd.DataFrame({
    'asset': mean_returns.index,
    'estimated_return': mean_returns.values
})


estimated_returns_df = estimated_returns_df.sort_values('estimated_return')

print("\nEstimated returns (formatted for saving):")
print(estimated_returns_df)

print("\nInterpretation:")
for _, row in estimated_returns_df.iterrows():
    asset = row['asset']
    return_val = row['estimated_return']
    annual_return = return_val * 12  # Convert monthly to approximate annual
    print(f"{asset}: {return_val:.6f} monthly ({annual_return:.2%} annualized)")

Historical returns data loaded:
Shape: (23, 7)
         date      AAPL       SPY      TSLA      YETI      MSFT      NVDA
0  2023-11-30  0.113780  0.091344  0.195379  0.002822  0.122945  0.146886
1  2023-12-31  0.013583  0.045655  0.034988  0.214353 -0.007574  0.058934
2  2024-01-31 -0.042227  0.015926 -0.246257 -0.150830  0.057281  0.242417
3  2024-02-29 -0.018543  0.052187  0.077901 -0.066636  0.042318  0.285809
4  2024-03-31 -0.051286  0.032702 -0.129235 -0.060673  0.017116  0.142181

Stock columns: ['AAPL', 'SPY', 'TSLA', 'YETI', 'MSFT', 'NVDA']

Calculated mean returns:
AAPL    0.016670
SPY     0.020767
TSLA    0.034135
YETI   -0.002113
MSFT    0.020324
NVDA    0.070501
dtype: float64

Estimated returns (formatted for saving):
  asset  estimated_return
3  YETI         -0.002113
0  AAPL          0.016670
4  MSFT          0.020324
1   SPY          0.020767
2  TSLA          0.034135
5  NVDA          0.070501

Interpretation:
YETI: -0.002113 monthly (-2.54% annualized)
AAPL: 0.016670 m

Save the estimated returns in a TSV file named "estimated_returns.tsv" and include a header row with the column names "asset" and "estimated_return".

In [20]:
# YOUR CHANGES HERE

try:
    
    estimated_returns_df.to_csv('estimated_returns.tsv', sep='\t', index=False, float_format='%.6f')
    print("Estimated returns saved to 'estimated_returns.tsv'")
    
    import os
    if os.path.exists('estimated_returns.tsv'):
        file_size = os.path.getsize('estimated_returns.tsv')
        print(f" File created successfully")
        print(f" File size: {file_size} bytes")
        
        print(f"\nFile contents:")
        saved_data = pd.read_csv('estimated_returns.tsv', sep='\t')
        print(saved_data)
    else:
        print(" Error: File was not created")
        
except NameError:
    print(" Error: 'estimated_returns_df' variable not found!")
    print("Please run the calculation cell first (Part 3 - Estimate Expected Returns)")
except Exception as e:
    print(f" Error saving file: {e}")

Estimated returns saved to 'estimated_returns.tsv'
 File created successfully
 File size: 107 bytes

File contents:
  asset  estimated_return
0  YETI         -0.002113
1  AAPL          0.016670
2  MSFT          0.020324
3   SPY          0.020767
4  TSLA          0.034135
5  NVDA          0.070501


Submit "estimated_returns.tsv" in Gradescope.

## Part 4: Estimate Risk

Estimate the covariance matrix for the asset returns to understand how the assets move together.

In [21]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

historical_returns_df = pd.read_csv('historical_returns.tsv', sep='\t')

print("Historical returns data loaded:")
print(f"Shape: {historical_returns_df.shape}")
print(historical_returns_df.head())

stock_columns = [col for col in historical_returns_df.columns if col != 'date']
print(f"\nStock columns: {stock_columns}")

returns_data = historical_returns_df[stock_columns]
covariance_matrix = returns_data.cov()

print(f"\nCovariance Matrix:")
print(f"Shape: {covariance_matrix.shape}")
print(covariance_matrix)

print(f"\nCovariance Matrix Interpretation:")
print(f"Diagonal elements (variances):")
for asset in stock_columns:
    variance = covariance_matrix.loc[asset, asset]
    volatility = np.sqrt(variance)
    print(f"  {asset}: {variance:.6f} (volatility: {volatility:.4f})")

print(f"\nSome key correlations:")
correlation_matrix = returns_data.corr()
print(f"AAPL-SPY correlation: {correlation_matrix.loc['AAPL', 'SPY']:.4f}")
print(f"TSLA-NVDA correlation: {correlation_matrix.loc['TSLA', 'NVDA']:.4f}")
print(f"SPY-MSFT correlation: {correlation_matrix.loc['SPY', 'MSFT']:.4f}")

Historical returns data loaded:
Shape: (23, 7)
         date      AAPL       SPY      TSLA      YETI      MSFT      NVDA
0  2023-11-30  0.113780  0.091344  0.195379  0.002822  0.122945  0.146886
1  2023-12-31  0.013583  0.045655  0.034988  0.214353 -0.007574  0.058934
2  2024-01-31 -0.042227  0.015926 -0.246257 -0.150830  0.057281  0.242417
3  2024-02-29 -0.018543  0.052187  0.077901 -0.066636  0.042318  0.285809
4  2024-03-31 -0.051286  0.032702 -0.129235 -0.060673  0.017116  0.142181

Stock columns: ['AAPL', 'SPY', 'TSLA', 'YETI', 'MSFT', 'NVDA']

Covariance Matrix:
Shape: (6, 6)
          AAPL       SPY      TSLA      YETI      MSFT      NVDA
AAPL  0.003615  0.000789  0.003052  0.002181  0.000422  0.000811
SPY   0.000789  0.001225  0.002006  0.001676  0.001596  0.002655
TSLA  0.003052  0.002006  0.024386  0.005601  0.002744 -0.001649
YETI  0.002181  0.001676  0.005601  0.010040  0.001571  0.001293
MSFT  0.000422  0.001596  0.002744  0.001571  0.003953  0.005289
NVDA  0.000811  0.002

Save the estimated covariances to a TSV file named "estimated_covariance.tsv".
The header row should have a blank column name followed by the names of the assets.
Each data row should start with the name of an asset for that row, and be followed by the individual covariances corresponding to that row and column's assets.
(This is the format of pandas's `to_csv` method with `sep="\t"` when used on a covariance matrix as computed in the examples.)

In [22]:
# YOUR CHANGES HERE

covariance_matrix.to_csv('estimated_covariance.tsv', sep='\t', float_format='%.6f')
print("Covariance matrix saved to 'estimated_covariance.tsv'")

Covariance matrix saved to 'estimated_covariance.tsv'


Submit "estimated_covariance.tsv" in Gradescope.

## Part 5: Construct the Maximum Return Portfolio

Compute the maximum return portfolio based on your previously estimated risks and returns.

In [23]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

estimated_returns_df = pd.read_csv('estimated_returns.tsv', sep='\t')

print("Estimated returns data:")
print(estimated_returns_df)

max_return_idx = estimated_returns_df['estimated_return'].idxmax()
best_asset = estimated_returns_df.loc[max_return_idx, 'asset']
max_return_value = estimated_returns_df.loc[max_return_idx, 'estimated_return']

print(f"\nAsset with highest expected return:")
print(f"Asset: {best_asset}")
print(f"Expected return: {max_return_value:.6f}")

all_assets = estimated_returns_df['asset'].tolist()
allocations = []

for asset in all_assets:
    if asset == best_asset:
        allocation = 1.0  
    else:
        allocation = 0.0  
    allocations.append(allocation)

max_return_portfolio = pd.DataFrame({
    'asset': all_assets,
    'allocation': allocations
})

print(f"\nMaximum Return Portfolio:")
print(max_return_portfolio)

total_allocation = max_return_portfolio['allocation'].sum()
print(f"\nTotal allocation: {total_allocation:.6f} (should be 1.0)")

portfolio_return = (max_return_portfolio['allocation'] * 
                   estimated_returns_df['estimated_return']).sum()
print(f"Portfolio expected return: {portfolio_return:.6f}")

Estimated returns data:
  asset  estimated_return
0  YETI         -0.002113
1  AAPL          0.016670
2  MSFT          0.020324
3   SPY          0.020767
4  TSLA          0.034135
5  NVDA          0.070501

Asset with highest expected return:
Asset: NVDA
Expected return: 0.070501

Maximum Return Portfolio:
  asset  allocation
0  YETI         0.0
1  AAPL         0.0
2  MSFT         0.0
3   SPY         0.0
4  TSLA         0.0
5  NVDA         1.0

Total allocation: 1.000000 (should be 1.0)
Portfolio expected return: 0.070501


Save the maximum return portfolio in a TSV file named "maximum_return.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [24]:
# YOUR CHANGES HERE

max_return_portfolio.to_csv('maximum_return.tsv', sep='\t', index=False, float_format='%.6f')
print("Maximum return portfolio saved to 'maximum_return.tsv'")

Maximum return portfolio saved to 'maximum_return.tsv'


Submit "maximum_return.tsv" in Gradescope.

## Part 6: Construct the Minimum Risk Portfolio

Compute the minimum return portfolio based on your previously estimated risks and returns.

In [25]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

covariance_matrix = pd.read_csv('estimated_covariance.tsv', sep='\t', index_col=0)

print("Covariance matrix loaded:")
print(f"Shape: {covariance_matrix.shape}")
print(covariance_matrix)

assets = covariance_matrix.index.tolist()
n_assets = len(assets)
print(f"\nAssets: {assets}")
print(f"Number of assets: {n_assets}")

cov_matrix = covariance_matrix.values

try:
    
    cov_inv = np.linalg.inv(cov_matrix)
   
    ones = np.ones((n_assets, 1))
    
    numerator = cov_inv @ ones
    
    denominator = ones.T @ cov_inv @ ones
    
    min_risk_weights = numerator / denominator
    min_risk_weights = min_risk_weights.flatten()  
    
    print(f"\nMinimum Risk Portfolio Weights:")
    for i, asset in enumerate(assets):
        print(f"{asset}: {min_risk_weights[i]:.6f}")
    
    total_weight = np.sum(min_risk_weights)
    print(f"\nTotal weights: {total_weight:.6f} (should be 1.0)")
    
    portfolio_variance = min_risk_weights.T @ cov_matrix @ min_risk_weights
    portfolio_risk = np.sqrt(portfolio_variance)
    print(f"Portfolio risk (std dev): {portfolio_risk:.6f}")
    
    min_risk_portfolio = pd.DataFrame({
        'asset': assets,
        'allocation': min_risk_weights
    })
    
    print(f"\nMinimum Risk Portfolio:")
    print(min_risk_portfolio)
    
except np.linalg.LinAlgError as e:
    print(f"Error: Covariance matrix is not invertible: {e}")
    print("This may indicate highly correlated assets or numerical issues.")

Covariance matrix loaded:
Shape: (6, 6)
          AAPL       SPY      TSLA      YETI      MSFT      NVDA
AAPL  0.003615  0.000789  0.003052  0.002181  0.000422  0.000811
SPY   0.000789  0.001225  0.002006  0.001676  0.001596  0.002655
TSLA  0.003052  0.002006  0.024386  0.005601  0.002744 -0.001649
YETI  0.002181  0.001676  0.005601  0.010040  0.001571  0.001293
MSFT  0.000422  0.001596  0.002744  0.001571  0.003953  0.005289
NVDA  0.000811  0.002655 -0.001649  0.001293  0.005289  0.014091

Assets: ['AAPL', 'SPY', 'TSLA', 'YETI', 'MSFT', 'NVDA']
Number of assets: 6

Minimum Risk Portfolio Weights:
AAPL: 0.126605
SPY: 1.151507
TSLA: -0.081720
YETI: -0.080394
MSFT: 0.084251
NVDA: -0.200250

Total weights: 1.000000 (should be 1.0)
Portfolio risk (std dev): 0.028542

Minimum Risk Portfolio:
  asset  allocation
0  AAPL    0.126605
1   SPY    1.151507
2  TSLA   -0.081720
3  YETI   -0.080394
4  MSFT    0.084251
5  NVDA   -0.200250


Save the minimum risk portfolio in a TSV file named "minimum_risk.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [26]:
# YOUR CHANGES HERE

min_risk_portfolio.to_csv('minimum_risk.tsv', sep='\t', index=False, float_format='%.6f')
print("Minimum risk portfolio saved to 'minimum_risk.tsv'")


Minimum risk portfolio saved to 'minimum_risk.tsv'


Submit "minimum_risk.tsv" in Gradescope.

## Part 7: Build Efficient Frontier Portfolios

Compute 101 portfolios along the mean-variance efficient frontier with evenly spaced estimated returns.
The first portfolio should be the minimum risk portfolio from part 4, and the last portfolio should be the maximum return portfolio from part 3.
The estimated return of each portfolio should be higher than the previous by one percent of the difference between the first and last portfolios.
That is, the estimated return of the portfolios should be similar to `np.linspace(min_risk_return, max_return, 101)`.


In [ ]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

covariance_matrix = pd.read_csv('estimated_covariance.tsv', sep='\t', index_col=0)
estimated_returns_df = pd.read_csv('estimated_returns.tsv', sep='\t')
min_risk_portfolio = pd.read_csv('minimum_risk.tsv', sep='\t')
max_return_portfolio = pd.read_csv('maximum_return.tsv', sep='\t')

print("Data loaded successfully")
print(f"Assets: {list(covariance_matrix.index)}")

assets = list(covariance_matrix.index)
n_assets = len(assets)
cov_matrix = covariance_matrix.values

expected_returns = np.zeros(n_assets)
for i, asset in enumerate(assets):
    expected_returns[i] = estimated_returns_df[estimated_returns_df['asset'] == asset]['estimated_return'].iloc[0]

print(f"\nExpected returns vector: {expected_returns}")

min_risk_weights = min_risk_portfolio['allocation'].values
max_return_weights = max_return_portfolio['allocation'].values

min_risk_weights_ordered = np.zeros(n_assets)
max_return_weights_ordered = np.zeros(n_assets)

for i, asset in enumerate(assets):
    min_idx = min_risk_portfolio[min_risk_portfolio['asset'] == asset].index[0]
    max_idx = max_return_portfolio[max_return_portfolio['asset'] == asset].index[0]
    min_risk_weights_ordered[i] = min_risk_portfolio.loc[min_idx, 'allocation']
    max_return_weights_ordered[i] = max_return_portfolio.loc[max_idx, 'allocation']

min_risk_return = np.dot(min_risk_weights_ordered, expected_returns)
max_return_return = np.dot(max_return_weights_ordered, expected_returns)

print(f"Min risk portfolio return: {min_risk_return:.6f}")
print(f"Max return portfolio return: {max_return_return:.6f}")

target_returns = np.linspace(min_risk_return, max_return_return, 101)
print(f"Target returns range: {target_returns[0]:.6f} to {target_returns[-1]:.6f}")

def solve_efficient_portfolio(target_return, expected_returns, cov_matrix):
    """
    Solve for minimum variance portfolio with target return
    Using the formula for constrained optimization
    """
    n = len(expected_returns)
    
    
    cov_inv = np.linalg.inv(cov_matrix)
    

    A = np.column_stack([expected_returns, np.ones(n)])  
    b = np.array([target_return, 1.0])  
    
   
    temp1 = A.T @ cov_inv @ A  
    temp2 = np.linalg.inv(temp1)  
    temp3 = temp2 @ b  
    weights = cov_inv @ A @ temp3  
    
    return weights

print(f"\nBuilding {len(target_returns)} efficient portfolios...")

efficient_portfolios = []

for i, target_ret in enumerate(target_returns):
    try:
        if i == 0:
            weights = min_risk_weights_ordered  
        elif i == len(target_returns) - 1:
            weights = max_return_weights_ordered
        else:
            weights = solve_efficient_portfolio(target_ret, expected_returns, cov_matrix)
        
        portfolio_return = np.dot(weights, expected_returns)
        portfolio_variance = weights.T @ cov_matrix @ weights
        portfolio_risk = np.sqrt(portfolio_variance)
        
        portfolio_data = {
            'index': i,
            'return': portfolio_return,
            'risk': portfolio_risk
        }
        
        for j, asset in enumerate(assets):
            portfolio_data[asset] = weights[j]
        
        efficient_portfolios.append(portfolio_data)
        
        if i % 20 == 0: 
            print(f"Portfolio {i}: Return = {portfolio_return:.6f}, Risk = {portfolio_risk:.6f}")
            
    except Exception as e:
        print(f"Error solving portfolio {i} with target return {target_ret:.6f}: {e}")

efficient_frontier_df = pd.DataFrame(efficient_portfolios)

print(f"\nEfficient frontier completed!")
print(f"Shape: {efficient_frontier_df.shape}")
print(f"Columns: {list(efficient_frontier_df.columns)}")
print(f"\nFirst few portfolios:")
print(efficient_frontier_df[['index', 'return', 'risk']].head())
print(f"\nLast few portfolios:")
print(efficient_frontier_df[['index', 'return', 'risk']].tail())

Data loaded successfully
Assets: ['AAPL', 'SPY', 'TSLA', 'YETI', 'MSFT', 'NVDA']

Expected returns vector: [ 0.01667   0.020767  0.034135 -0.002113  0.020324  0.070501]
Min risk portfolio return: 0.010999
Max return portfolio return: 0.070501
Target returns range: 0.010999 to 0.070501

Building 101 efficient portfolios...
Portfolio 0: Return = 0.010999, Risk = 0.028542
Portfolio 20: Return = 0.022899, Risk = 0.032744
Portfolio 40: Return = 0.034800, Risk = 0.042950
Portfolio 60: Return = 0.046700, Risk = 0.055967
Portfolio 80: Return = 0.058601, Risk = 0.070249
Portfolio 100: Return = 0.070501, Risk = 0.118706

Efficient frontier completed!
Shape: (101, 9)
Columns: ['index', 'return', 'risk', 'AAPL', 'SPY', 'TSLA', 'YETI', 'MSFT', 'NVDA']

First few portfolios:
   index    return      risk
0      0  0.010999  0.028542
1      1  0.011594  0.028553
2      2  0.012189  0.028587
3      3  0.012784  0.028643
4      4  0.013379  0.028721

Last few portfolios:
     index    return      risk
9

Save the portfolios in a TSV file named "efficient_frontier.tsv".
The header row should have columns "index", "return", "risk", and all the asset tickers.
Each data row should have the portfolio index (0-100), the estimated return of the portfolio, the estimated standard deviation (not variance) of the portfolio, and all the asset allocations (which should sum to one).

In [28]:
# YOUR CHANGES HERE

efficient_frontier_df.to_csv('efficient_frontier.tsv', sep='\t', index=False, float_format='%.6f')
print("Efficient frontier saved to 'efficient_frontier.tsv'")

Efficient frontier saved to 'efficient_frontier.tsv'


Submit "efficient_frontier.tsv" in Gradescope.

## Part 8: Check Maximum Return Portfolio Stability

Check the stability of the maximum return portfolio by resampling the estimated risk/return model.

Repeat 1000 times -
1. Use `np.random.multivariate_normal` to generate 23 return samples using your previously estimated risks and returns.
2. Estimate the return of each asset using that resampled return history.
3. Check which asset had the highest return in those resampled estimates.

This procedure is a reduced and simplified version of the Michaud resampled efficient frontier procedure that takes uncertainty in the risk model into account.

In [29]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

covariance_matrix = pd.read_csv('estimated_covariance.tsv', sep='\t', index_col=0)
estimated_returns_df = pd.read_csv('estimated_returns.tsv', sep='\t')

print("Data loaded for stability check")

assets = list(covariance_matrix.index)
n_assets = len(assets)
cov_matrix = covariance_matrix.values

expected_returns = np.zeros(n_assets)
for i, asset in enumerate(assets):
    expected_returns[i] = estimated_returns_df[estimated_returns_df['asset'] == asset]['estimated_return'].iloc[0]

print(f"Assets: {assets}")
print(f"Original expected returns: {expected_returns}")

original_max_asset = estimated_returns_df.loc[estimated_returns_df['estimated_return'].idxmax(), 'asset']
print(f"Original maximum return asset: {original_max_asset}")

n_simulations = 1000
n_samples = 23  

print(f"\nRunning {n_simulations} Monte Carlo simulations...")
print(f"Each simulation generates {n_samples} return samples")

highest_return_assets = []

np.random.seed(42)

for sim in range(n_simulations):
    simulated_returns = np.random.multivariate_normal(
        mean=expected_returns, 
        cov=cov_matrix, 
        size=n_samples
    )
    
    sim_mean_returns = np.mean(simulated_returns, axis=0)
    
    max_return_idx = np.argmax(sim_mean_returns)
    highest_return_asset = assets[max_return_idx]
    highest_return_assets.append(highest_return_asset)
   
    if (sim + 1) % 200 == 0:
        print(f"Completed {sim + 1} simulations...")

print(f"\nSimulation completed!")

from collections import Counter
asset_counts = Counter(highest_return_assets)

print(f"\nResults summary:")
print(f"Asset with highest return in each simulation:")
for asset in assets:
    count = asset_counts.get(asset, 0)
    probability = count / n_simulations
    print(f"{asset}: {count} times ({probability:.3f} probability)")

results_df = pd.DataFrame({
    'asset': assets,
    'probability': [asset_counts.get(asset, 0) / n_simulations for asset in assets]
})

print(f"\nStability analysis:")
original_prob = results_df[results_df['asset'] == original_max_asset]['probability'].iloc[0]
print(f"Original choice ({original_max_asset}) was optimal in {original_prob:.1%} of simulations")

if original_prob < 0.5:
    print(f"⚠ Warning: Original choice was not optimal in majority of simulations")
    print(f"This suggests the maximum return portfolio choice is not very stable")
else:
    print(f"✓ Original choice was optimal in majority of simulations")
    print(f"This suggests the maximum return portfolio choice is relatively stable")

results_sorted = results_df.sort_values('probability', ascending=False)
print(f"\nAssets ranked by probability of having highest return:")
print(results_sorted)

Data loaded for stability check
Assets: ['AAPL', 'SPY', 'TSLA', 'YETI', 'MSFT', 'NVDA']
Original expected returns: [ 0.01667   0.020767  0.034135 -0.002113  0.020324  0.070501]
Original maximum return asset: NVDA

Running 1000 Monte Carlo simulations...
Each simulation generates 23 return samples
Completed 200 simulations...
Completed 400 simulations...
Completed 600 simulations...
Completed 800 simulations...
Completed 1000 simulations...

Simulation completed!

Results summary:
Asset with highest return in each simulation:
AAPL: 2 times (0.002 probability)
SPY: 1 times (0.001 probability)
TSLA: 186 times (0.186 probability)
YETI: 1 times (0.001 probability)
MSFT: 0 times (0.000 probability)
NVDA: 810 times (0.810 probability)

Stability analysis:
Original choice (NVDA) was optimal in 81.0% of simulations
✓ Original choice was optimal in majority of simulations
This suggests the maximum return portfolio choice is relatively stable

Assets ranked by probability of having highest return

Save a file "max_return_probabilities.tsv" with the distribution of highest return assets.
The header row should have columns "asset" and "probability".
There should be a data row for each asset and its sample probability of having the highest return based on those 1000 resampled estimates.


In [30]:
# YOUR CHANGES HERE

results_df.to_csv('max_return_probabilities.tsv', sep='\t', index=False, float_format='%.6f')
print("Maximum return portfolio stability results saved to 'max_return_probabilities.tsv'")

Maximum return portfolio stability results saved to 'max_return_probabilities.tsv'


Submit "max_return_probabilities.tsv" in Gradescope.

## Part 9: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgements.txt" in Gradescope.

## Part 10: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.